In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import load_files
from nltk import pos_tag
import json
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from nltk.corpus import stopwords #import the stopwords from the ntlk.corpus library
nltk.download('stopwords')
from nltk.tokenize import word_tokenize #import the word_tokenize method, which is used to turn sentences into words"
import re #import the regular expressions library
nltk.download('punkt')
from collections import Counter
from nltk.stem import PorterStemmer 
import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:



# import spacy

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
FullPaper=[]
directories = ["biorxiv_medrxiv"]
for directory in directories: #for each of the three folders carrying the json format of different research papers
    for file in tqdm(os.listdir(f"{directory}/{directory}/pdf_json")): #for every json file
        file_path=f"{directory}/{directory}/pdf_json/{file}" #set the file path to the file_path variable 
        paper = json.load(open(file_path,"rb"))
        title = paper['metadata']['title'] 
        try:
            abstract = paper['abstract']
        except:
            abstarct=""
                
        full_text=""
            
        for text in paper['body_text']:
            full_text +=text['text'] +'\n\n' 
            FullPaper.append([title,abstract,full_text])
            
FullPaperDataframe=pd.DataFrame(FullPaper,columns=['title','abstract','full_text'])

# Getting all the papers related to vaccine

# vaccine=FullPaperDataframe[FullPaperDataframe['full_text'].str.contains('vaccine')]

# for para in vaccine['full_text']:
#     for sentence in para.split('. '):
#         if 'vaccine' in sentence:
#             print(sentence)


100%|██████████| 1934/1934 [00:51<00:00, 37.42it/s]


> Our aim is to address these points:
* Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.

In [40]:
full_text = FullPaperDataframe['full_text'].head()
clean_text = []
ps=PorterStemmer()   
#cleaning the data and removing stop words
for val in  tqdm(full_text):
        body_tokens = word_tokenize(val)
        paper_body_without_stopwords = [token for token in body_tokens if not token in stopwords.words('english')] #remove the stop words in the body and return a list
        clean_string = ' '.join(paper_body_without_stopwords) #convert the list into string
        remove_punc = re.sub(r"[^\P{P}-]+", " ", clean_string)
        final_cleaned_String = re.sub(r"\bThe\b", r"", remove_punc)
        clean_text.append(final_cleaned_String) #add the string to the array 
stemmed_text=[]       
for val in clean_text:
    stemned_val=""
    for word in val:
        stemned_val+=ps.stem(word)
    stemmed_text.append(stemned_val)
text = ' '.join(stemmed_text)      
s=text.split(' ')
r = Counter(s)
most_occur = r.most_common(50) 
print(most_occur)

# finding the tf-idf matrix for all the abstracts
# vec = TfidfVectorizer()
# ve = vec.fit_transform(clean_abs)

# # displaying the tf-idf of the word in the abstact  
# pd.DataFrame(ve.toarray(), columns=sorted(vec.vocabulary_.keys()))

# #applying tf-idf cosine similarity and printing the result(testing)
# query = ["prophylaxis"]
# query_tfidf = vec.transform(query)
# cosineSimilarities = cosine_similarity(query_tfidf, ve).flatten()
# print(cosineSimilarities)
    


  0%|          | 0/5 [00:00<?, ?it/s]


error: bad escape \P at position 2

In [ ]:
taged_abs=[]
nouns=[]
# tagging the clean data
for val in clean_abs:
    taged_abs.append(pos_tag(val.split()))
# extracting the nouns
for i in range(len(taged_abs)):
    doc=[]
    for j in range(len(taged_abs[i])):
        if(taged_abs[i][j][1]=='NN' or taged_abs[i][j][1]=='NNS'):
            doc.append(taged_abs[i][j][0])
    nouns.append(doc)
        
print(nouns)



Above we dcided to calculate the tf-idf so that we can represent every word that is present in the abstract quatitavily. By doing so we can further use the results in order model the topic according to the abstract that we just quatified.Furthermore we will use (Non-negative Matrix Factorization) NMF in order to come up with topic's that carry most weight in the abstract. To accomplish this we are going to filter all the nouns that are avaliable in the abstract and use them to represent the different topis that are avaliable.